In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\test\cats\cat_1.jpg
input\test\cats\cat_106.jpg
input\test\cats\cat_109.jpg
input\test\cats\cat_113.jpg
input\test\cats\cat_114.jpg
input\test\cats\cat_116.jpg
input\test\cats\cat_118.jpg
input\test\cats\cat_119.jpg
input\test\cats\cat_124.jpg
input\test\cats\cat_140.jpg
input\test\cats\cat_147.jpg
input\test\cats\cat_156.jpg
input\test\cats\cat_158.jpg
input\test\cats\cat_162.jpg
input\test\cats\cat_18.jpg
input\test\cats\cat_190.jpg
input\test\cats\cat_203.jpg
input\test\cats\cat_223.jpg
input\test\cats\cat_234.jpg
input\test\cats\cat_244.jpg
input\test\cats\cat_251.jpg
input\test\cats\cat_255.jpg
input\test\cats\cat_268.jpg
input\test\cats\cat_279.jpg
input\test\cats\cat_281.jpg
input\test\cats\cat_290.jpg
input\test\cats\cat_306.jpg
input\test\cats\cat_313.jpg
input\test\cats\cat_332.jpg
input\test\cats\cat_341.jpg
input\test\cats\cat_342.jpg
input\test\cats\cat_355.jpg
input\test\cats\cat_358.jpg
input\test\cats\cat_371.jpg
input\test\cats\cat_375.jpg
input\test\cats\cat_384

In [34]:
import os 
import cv2 
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

seed = 42

In [35]:
train_folder = "input/train"
test_folder = "input/test"
print("Training Folder path:  ",train_folder)
print("Testing Folder path:   ",test_folder)

Training Folder path:   input/train
Testing Folder path:    input/test


In [36]:
print(os.listdir(train_folder))
for subfolder in os.listdir(train_folder):
    print('subfolder name: ',subfolder)
subfolder_path = os.path.join(train_folder, subfolder)
print('subfolder path: ',subfolder_path)

['cats', 'dogs', '__MACOSX']
subfolder name:  cats
subfolder name:  dogs
subfolder name:  __MACOSX
subfolder path:  input/train\__MACOSX


In [38]:
def load_images(folder):
    images = []  
    labels = []  

    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)

        if subfolder == 'dogs':
            label = 1
        else:
            label = 0

        for imagelist in os.listdir(subfolder_path):
            if imagelist.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(subfolder_path, imagelist)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (64, 64))
                images.append(image)
                labels.append(label)

    return np.array(images), np.array(labels)

In [39]:
train_images, train_labels = load_images(train_folder)
test_images, test_labels   = load_images(test_folder)

In [40]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(train_labels.shape)

(557, 64, 64, 3)
(557,)
(140, 64, 64, 3)
(557,)


In [41]:
flat_train_images  = train_images.reshape(train_images.shape[0], -1) 
flat_test_images   = test_images.reshape(test_images.shape[0], -1)

print(flat_train_images.shape)
print(flat_test_images.shape)

(557, 12288)
(140, 12288)


In [42]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create and train the SVM model
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=seed))
svm_model.fit(flat_train_images, train_labels)

# Make predictions on the test set
svm_predictions = svm_model.predict(flat_test_images)

# Calculate accuracy
accuracy_svm = accuracy_score(test_labels, svm_predictions)
print("Accuracy Score of SVM model:", accuracy_svm)

Accuracy Score of SVM model: 0.5714285714285714


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# Define the SVM model with a pipeline for feature scaling
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=seed))

# Define a parameter grid for GridSearchCV
param_grid = {
    'svc__C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'svc__gamma': ['scale', 'auto', 0.1, 1, 10],  # Kernel coefficient
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(svm_model, param_grid, cv=5)
grid_search.fit(flat_train_images, train_labels)
best_svm_model = grid_search.best_estimator_

# Make predictions on the test set
svm_predictions = best_svm_model.predict(flat_test_images)

# Calculate accuracy
accuracy_svm = accuracy_score(test_labels, svm_predictions)
print("Accuracy Score of SVM model:", accuracy_svm)
print("Best hyperparameters:", grid_search.best_params_)